In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# PROBLEM STATEMENT 1.1
df = pd.read_csv('PVR.csv')

FileNotFoundError: File b'PVR.csv' does not exist

In [95]:
df.tail()

,Date,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),Month,Year,PV
489,2019-05-07,11651.50,11657.05,11484.45,11497.90,337495624,16632.84,May,2019,3.880491e+12
490,2019-05-08,11478.70,11479.10,11346.95,11359.45,372826025,17440.01,May,2019,4.235099e+12
491,2019-05-09,11322.40,11357.60,11255.05,11301.80,373028059,17602.86,May,2019,4.215889e+12
492,2019-05-10,11314.15,11345.80,11251.05,11278.90,387323416,18085.19,May,2019,4.368582e+12
493,2019-05-13,11258.70,11300.20,11125.60,11148.20,357586433,16722.91,May,2019,3.986445e+12


In [78]:
# PROBLEM STATEMENT 1.2
# Calculate the max., min. and the mean price of the stock in the last 90 trading days
df_last_90_days = df.tail(90)
last_90_close = df_last_90_days['Close']

min_90 = last_90_close.min()
max_90 = last_90_close.max()
mean_90 = last_90_close.mean()

print('min : ' + str(min_90))
print('max : ' + str(max_90))
print('mean : ' + str(mean_90))

min : 10604.35
max : 11787.15
mean : 11160.763333333332


In [98]:
# PROBLEM STATEMENT 1.3
# Change the dtype of the 'Date' column to datetime

import datetime as dt

# df['Date'] = df['Date'].apply(lambda x : dt.datetime.strptime(x, '%d-%b-%Y'))

In [80]:
# Subtracting min value of 'Date' from max value
print(df['Date'].max() - df['Date'].min())

728 days 00:00:00


In [82]:
# PROBLEM STATEMENT 1.4
# Calculate monthwise VWAP of the stock.

df['Month'] = df['Date'].apply(lambda x : datetime.strftime(x, '%B'))
df['Year'] = df['Date'].apply(lambda x : datetime.strftime(x, '%Y'))

# To calculate the VWAP we will also add a 'PV' columns 
# which gives us the "price * volume" for that day
df['PV'] = df['Close'] * df['Shares Traded']

# Calculate VWAP
df_group = df.groupby(['Year', 'Month']).sum()

VWAP = df_group['PV'] / df_group['Shares Traded'] 
print(VWAP)

In [146]:
# PROBLEM STATEMENT 1.5
# (a) Write a function to calculate average stock price over last N days
# (b) Write a funciton to calculate profit/loss percentage over last N days

# Note : The values will be calculated assuming the current date to be the last date in the dataset (i.e. 13-May-2019)

# Function (a)
def avg_over_days(N):
    delta = dt.timedelta(days = N)
    df_filtered = df[df['Date'] >= df.loc[df.shape[0]-1]['Date'] - delta]['Close']
    return df_filtered.mean()

# Function (b)
def profit_loss_perc_over_days(N):
    delta = dt.timedelta(days = N)
    df_filtered = df_filtered = df[df['Date'] >= df.iloc[-1]['Date'] - delta]['Close']
    
    # using formula : 100 * (final_price - initial_price) / initial_price
    return 100 * (df_filtered.iloc[-1] - df_filtered.iloc[0]) / df_filtered.iloc[0]


print('Avg. price over 7 days : {0}, Profit % : {1}'.format(avg_over_days(7), profit_loss_perc_over_days(7)))
print('Avg. price over 2 weeks : {0}, Profit % : {1}'.format(avg_over_days(14), profit_loss_perc_over_days(14)))
print('Avg. price over 1 month : {0}, Profit % : {1}'.format(avg_over_days(30), profit_loss_perc_over_days(30)))
print('Avg. price over 3 months : {0}, Profit % : {1}'.format(avg_over_days(90), profit_loss_perc_over_days(90)))
print('Avg. price over 6 months : {0}, Profit % : {1}'.format(avg_over_days(180), profit_loss_perc_over_days(180)))
print('Avg. price over 1 year : {0}, Profit % : {1}'.format(avg_over_days(365), profit_loss_perc_over_days(365)))

Avg. price over 7 days : 11364.083333333336, Profit % : -3.8803267734356415
Avg. price over 2 weeks : 11485.516666666666, Profit % : -5.106761490106944
Avg. price over 1 month : 11581.938235294117, Profit % : -4.637585701026912
Avg. price over 3 months : 11326.847457627116, Profit % : 2.9248296619089045
Avg. price over 6 months : 11052.686776859504, Profit % : 5.407373088887432
Avg. price over 1 year : 10978.849186991867, Profit % : 3.1610312216608403


In [160]:
# PROBLEM STATEMENT 1.6
# Add column 'Day_perc_change'

df['Day_Perc_Change'] = df['Close'].pct_change()

# set the first value to 0 because it is NaN. It is NaN because 2 values are required to
# calculate the percentage but only 1 value is available for the first row
df.at[0,'Day_Perc_Change'] = 0

In [162]:
# PROBLEM STATEMENT 1.7
# Add a 'Trend' column to the dataset

def get_trend(change_perc):
    if change_perc < 0.5 and change_perc >= -0.5:
        return 'slight or no change'
    elif change_perc >= 0.5 and change_perc < 1:
        return 'slight positive'
    elif change_perc < -0.5 and change_perc >= -1:
        return 'slight negative'
    elif change_perc >= 1 and change_perc < 3:
        return 'positive'
    elif change_perc < -1 and change_perc >= -3:
        return 'negative'
    elif change_perc >= 3 and change_perc < 7:
        return 'among top gainers'
    elif change_perc < 3 and change_perc >= -7:
        return 'among top losers'
    elif change_perc >= 7:
        return 'bull run'
    elif change_perc < -7:
        return 'bear drop'
    
df['Trend'] = df['Day_Perc_Change'].apply(lambda x : get_trendnd(x))

In [166]:
# PROBLEM STATEMENT 1.8
df.groupby('Trend').mean()

,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),PV,Day_Perc_Change
Trend,,,,,,,,
slight or no change,10583.743016,10622.674291,10525.919939,10574.617308,2.648677e+08,13677.918806,2.823385e+12,0.000361
